In [ ]:
!pip install spacy
!pip install nltk

Imports

In [9]:
import sys
import os
import re
import spacy
import json
import nltk
from nltk.stem import RSLPStemmer

nltk.download("rslp", quiet=True)      # Stemmer para português

stemmer_pt = RSLPStemmer()


try:
    print("Modelo spaCy para português encontrado. Carregando...")
    nlp_pt = spacy.load("pt_core_news_lg")
except OSError:
    print("Modelo spaCy para português não encontrado. Fazendo download...")
    os.system("python -m spacy download pt_core_news_lg")
    nlp_pt = spacy.load("pt_core_news_lg")

try:
    with open('corpus_bracis.json', 'r', encoding='utf-8') as f:
        corpus = json.load(f)
except Exception as e:
    print(f"Erro ao abrir o arquivo JSON: {e}")
    sys.exit

print("✅ Concluido.")

Modelo spaCy para português encontrado. Carregando...
✅ Concluido.


Extrai tokens, pos-tags e lemas

In [11]:
def limpar_texto(texto):
    # Normaliza quebras de linha e espaços
    texto = re.sub(r'[\n\r\t]+', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    # Substituir caracteres Unicode problemáticos
    texto = texto.replace("−", "-").replace("—", "-").replace("–", "-")
    return texto.strip()

for estrutura_artigo_corpus in corpus:
    print("Fazendo extração do artigo: " + estrutura_artigo_corpus["titulo"])
    artigo_pt = limpar_texto(estrutura_artigo_corpus["artigo_completo_PT"])
    try:
        # Aumenta limite de texto para spaCy se necessário
        if len(artigo_pt) > nlp_pt.max_length:
            nlp_pt.max_length = len(artigo_pt) + 1000

        artigo_nlp = nlp_pt(artigo_pt)

        tokens = []
        pos_tags = []
        lemas = []
        
        for token in artigo_nlp:
            # Ignora pontuações e números
            if token.is_punct or token.is_digit or token.is_space:
                continue
                
            tokens.append(token.text)
            pos_tags.append(token.pos_)

            # Se o lema for igual ao token (spaCy não alterou), tenta com stemmer RSLP
            lema_spacy = limpar_str(token.lemma_)
            if lema_spacy != token.text:
                lemas.append(lema_spacy)
            else:
                try:
                    # Usa o stemmer RSLP como backup
                    lema_rslp = stemmer_pt.stem(token.text.lower())
                    lemas.append(lema_rslp)
                except:
                    lemas.append(token.text)
            
        estrutura_artigo_corpus["artigo_tokenizado"] = tokens
        estrutura_artigo_corpus["pos_tagger"] = pos_tags
        estrutura_artigo_corpus["lema"] = lemas

    except Exception as e:
        print(f"Erro ao processar com spaCy: {e}")
        sys.exit
        
with open('corpus_bracis.json', "w", encoding="utf-8") as f:
    json.dump(corpus, f, ensure_ascii=False, indent=2)
    
print("✅ Concluido.")

Fazendo extração do artigo: A Large Dataset of Spontaneous Speech with the Accent Spoken in São Paulo for Automatic Speech Recognition Evaluation
Fazendo extração do artigo: A Transformer-Based Tabular Approach to Detect Toxic Comments
Fazendo extração do artigo: An Ensemble of LLMs Finetuned with LoRA for NER in Portuguese Legal Documents
Fazendo extração do artigo: Aroeira: A Curated Corpus for the Portuguese Language with a Large Number of Tokens
Fazendo extração do artigo: Assessing European and Brazilian Portuguese LLMs for NER in Specialised Domains
Fazendo extração do artigo: Automatic Text Simplification for the Legal Domain in Brazilian Portuguese
Fazendo extração do artigo: Developing Resource-Efficient Clinical LLMs for Brazilian Portuguese
Fazendo extração do artigo: Diversity in Data for Speech Processing in Brazilian Portuguese
Fazendo extração do artigo: ERASMO: Leveraging Large Language Models for Enhanced Clustering Segmentation
Fazendo extração do artigo: Evaluating L